In [1]:
import torch
import json
from PIL import Image
import requests
from tqdm import tqdm
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration
import sys

In [5]:
image_path = '/scratch/nmachav1/CLEVR_v1.0/images/val/'
model = InstructBlipForConditionalGeneration.from_pretrained("Salesforce/instructblip-vicuna-7b")
processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-vicuna-7b")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

image_path = '/scratch/nmachav1/CLEVR_v1.0/images/val/'
#object_type = ['inter', 'intra']
for i in range(3,11):
    dataset = f'/scratch/nmachav1/MLLM_Hallucinations_CLEVR/dataset/object_number_split/val_num_{i}.json'
    output_path = f'/scratch/nmachav1/MLLM_Hallucinations_CLEVR/outputs/InstructBLIP/val_num_{i}.json'
    #dataset = f'/scratch/nmachav1/MLLM_Hallucinations_CLEVR/dataset/object_type_split/val_type_{i}.json'
    #output_path = f'/scratch/nmachav1/MLLM_Hallucinations_CLEVR/outputs/LLaVA/val_type_{i}.json'
    with open(dataset, 'r') as f:
        data = json.load(f)


    complete_data = []
    for each in tqdm(data[:1]):
        each_copy = each.copy()
        img = Image.open(image_path+each['image_filename']).convert("RGB")
        prompt = each['question']
        inputs = processor(images=img, text=prompt, return_tensors="pt").to(device)
        outputs = model.generate(
            **inputs,
            do_sample=False,
            use_cache=True,
            num_beams=5,
            max_length=256,
            min_length=1,
            top_p=1.0,
            repetition_penalty=1.0,
            length_penalty=1,
            top_k=50,
            temperature=1e-05
        )
        generated_text = processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()
        each_copy['model_generated_answer'] = generated_text
        print(each_copy)
        complete_data.append(each_copy)


    with open(output_path, 'w') as f:
        json.dump(complete_data, f ,indent=4)




Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:04<00:00,  4.92s/it]

{'image_filename': 'CLEVR_val_000006.png', 'question_index': 61, 'answer_type': 'yes-no', 'question': 'Is the shape of the small gray matte thing the same as the object behind the big green rubber object?', 'ground_truth': 'no', 'model_generated_answer': 'no', 'consistent': ''}



100%|██████████| 1/1 [00:02<00:00,  2.42s/it]

{'image_filename': 'CLEVR_val_000007.png', 'question_index': 71, 'answer_type': 'number', 'question': 'What number of objects are tiny spheres or brown blocks behind the gray matte object?', 'ground_truth': '2', 'model_generated_answer': 'There are three tiny spheres or brown blocks behind the gray matte object.', 'consistent': ''}


...

{'image_filename': 'CLEVR_val_000002.png', 'question_index': 21, 'answer_type': 'yes-no', 'question': 'Is the color of the metal block that is right of the yellow rubber object the same as the large metal cylinder?', 'ground_truth': 'yes', 'model_generated_answer': 'No', 'consistent': ''}



100%|██████████| 1/1 [00:12<00:00, 12.56s/it]

{'image_filename': 'CLEVR_val_000008.png', 'question_index': 80, 'answer_type': 'yes-no', 'question': 'Do the purple cylinder and the yellow rubber thing have the same size?', 'ground_truth': 'no', 'model_generated_answer': '10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000', 'consistent': ''}



100%|██████████| 1/1 [00:00<00:00,  1.83it/s]

{'image_filename': 'CLEVR_val_000001.png', 'question_index': 10, 'answer_type': 'colour', 'question': 'The other small shiny thing that is the same shape as the tiny yellow shiny object is what color?', 'ground_truth': 'cyan', 'model_generated_answer': 'blue', 'consistent': ''}
